# Relatório de Avaliação de Modelos para Armazenamento de Áudio em Bases de Dados Vetoriais (Milvus)

Este relatório apresenta uma avaliação comparativa (benchmark) de diferentes modelos utilizados processar e armazenar ficheiros de áudio em bases de dados vetoriais (Milvus). Ao longo deste notebook interativo, será possível analisar o desempenho de cada modelo/embedding, permitindo perceber qual deles se adapta melhor às nossas necessidades. No final, será possível eleger o modelo ou embedding mais adequado para o nosso caso específico.


## Introdução

## Falar o que fizemos para fazer o benchmark

## Fases do projeto

## Porque o milvus?

-TODO: Falar porque é que escolhemos o milvus

### Falar dos embeddings escolhidos

### Falar do script de benchmark...

## Análise dos resultados

Instalar dependências
```bash
pip install -r requirements.txt
```

```text
# requirements.txt

TODO ESCREVER O QUE ESTÁ NO REQUERIMENTS.TXT
```

PARA CORRER O NOTEBOOK É NECESSÁRIO TER CERTAS DEPÊNCIAS... ALSO O MILVUS NÃO VAI DAR... MAS É POSSIVEL FAZER A INSTALAÇÃO DO MILVUS FACILMENTE...
É SÓ CORRER O DOCKER COM O COMANDO ... 

TODO : VER FORMAS DE FAZER BUILD DO FICHEIRO DOCKER, PARA CONSEGUIR LEVANTAR A BASE DE DADOS... VER TAMBÉM SE OS FICHEIROS SÃO PERSISTENTES... EU ACHO QUE SIM


In [2]:
#!pip install -r requirements_jupyter.txt

### 1º Imports

In [3]:
# 📦 Importações e leitura dos dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.manifold import TSNE
import ast
from pymilvus import MilvusClient
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import io
from IPython.display import display
import ipywidgets as widgets




### 2º Carregar dados

DEPOIS COMO VAMOS TER ESCALIBIDADE VAMOS TER QUE ADICIONAR O RESTO DOS BENCHMARKS

In [4]:
# 📂 Leitura e limpeza dos dados
try:
    df = pd.read_csv('benchmarks/benchmark_30/resultados_benchmark.csv')
    df['embedding'] = df['embedding'].apply(ast.literal_eval)
    
    # Success message with dataset information
    print(f"✅ Dados carregados com sucesso!")
    print(f"   - {len(df)} registros de embeddings carregados")
    print(f"   - Modelos disponíveis: {', '.join(df['modelo'].unique())}")
    
except FileNotFoundError as e:
    print(f"❌ Erro ao carregar os arquivos de benchmark: {e}")
    print("   Verifique se os arquivos estão no diretório correto: benchmarks/benchmark_30/")
except Exception as e:
    print(f"❌ Erro ao processar os dados: {e}")
    print("   Verifique o formato dos arquivos CSV e tente novamente.")

✅ Dados carregados com sucesso!
   - 30 registros de embeddings carregados
   - Modelos disponíveis: wav2vec2, vggish, openl3, yamnet, clap, ast


### 3ºCarregar dados

In [5]:
# 🔄 Conexão com Milvus e seleção da coleção
try:
    # Tentar conectar ao Milvus
    client = MilvusClient(uri='http://localhost:19530')
    
    # Obter coleções disponíveis
    colecoes_disponiveis = client.list_collections()
    
    if colecoes_disponiveis:
        print(f"✅ Conexão com Milvus estabelecida com sucesso!")
        print(f"   - {len(colecoes_disponiveis)} coleções disponíveis")
        print(f"   - Coleções: {', '.join(colecoes_disponiveis)}")
        
    else:
        print("⚠️ Conectado ao Milvus, mas nenhuma coleção encontrada.")
        print("   Verifique se os dados foram carregados corretamente.")
        
except ConnectionError as e:
    print(f"❌ Erro de conexão com o Milvus: {e}")
    print("   Verifique se o servidor Milvus está em execução em http://localhost:19530")
    print("   Para iniciar o Milvus localmente com Docker, execute:")
    print("   docker-compose up -d")
    client = None
    
except Exception as e:
    print(f"❌ Erro ao conectar ao Milvus: {e}")
    print("   Verifique sua configuração e tente novamente.")
    client = None

✅ Conexão com Milvus estabelecida com sucesso!
   - 5 coleções disponíveis
   - Coleções: audio_clap_5, audio_vggish_5, audio_openl3_5, audio_yamnet_5, audio_ast_5


### 4º Analise dos resultados

### ANALISAR E EXPLICAR O QUE É QUE ESTÁ A ACONTECER.. EXPLICAR CADA UMA DAS TABELAS 

In [6]:
# 📊 Gráficos interativos por métrica em abas
tab_contents = ['Extração', 'Inserção', 'Boxplot Extração', 'Boxplot Inserção','Tamanho vs Extração','Tamanho vs Inserção' ]
children = []

fig1 = px.bar(df, x='modelo', y='tempo_extracao', color='modelo', title='Tempo de Extração por Modelo')
fig2 = px.bar(df, x='modelo', y='tempo_insercao', color='modelo', title='Tempo de Inserção por Modelo')
fig3 = px.box(df, x='modelo', y='tempo_extracao', color='modelo', title='Boxplot Tempo de Extração')
fig4 = px.box(df, x='modelo', y='tempo_insercao', color='modelo', title='Boxplot Tempo de Inserção')
fig5 = px.scatter(df, x='tamanho_audio_mb', y='tempo_extracao', color='modelo', title='Tamanho vs Extração')
fig6 = px.scatter(df, x='tamanho_audio_mb', y='tempo_insercao', color='modelo', title='Tamanho vs Inserção')

#ADICIONAR MAIS ? 



for fig in [fig1, fig2, fig3, fig4,fig5, fig6]:
    # Create a new tab for each figure
    out = widgets.Output()
    with out:
        display(fig)
    children.append(out)

tab = widgets.Tab(children=children)
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])
display(tab)

### 5º RANKINGS... TEMPO DE INSERÇÃO ETC... FAZER ISTO PARA CADA COLECTION... 5, 25, 50, 100, 200 FICHEIROS...

EXPLICAR COMO É QUE A ESCALIBIDADE INFLUENCIA... CHEGAR A UMA CONCLUSÃO FINAL

### 7º EXPLICAR EVOLUÇÃO DOS RECURSOS

In [7]:
# 📈 Evolução dos recursos (CPU, RAM, GPU)
recursos = ['cpu_percent_after', 'ram_percent_after', 'gpu_usage']
for recurso in recursos:
    fig = go.Figure()
    for modelo in df['modelo'].unique():
        subset = df[df['modelo'] == modelo]
        fig.add_trace(go.Scatter(x=subset.index, y=subset[recurso], mode='lines+markers', name=modelo))
    fig.update_layout(title=f'Evolução de {recurso.upper()}', xaxis_title='Index', yaxis_title=recurso)
    fig.show()

# FAZER RANKING DOS RECURSOS... CHEGAR A UMA CONCLUSÃO FINAL

EXPLICAR PARA A ESCALIBIDADE

### 8 VISUALIZAÇÕES DOS EMBEDDINGS NO ESPAÇO ... EXPLICAR O QUE SIGNIFICA SE ESTA MAIS PROXIMOS OU AFASTADOS UNS DOS OUTROS

In [8]:
# 🧠 Visualização t-SNE com validação
modelo_tsne = 'wav2vec2'
subset = df[df['modelo'] == modelo_tsne]
if len(subset) >= 2:
    try:
        emb = np.stack(subset['embedding'].values)
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(emb)-1))
        emb_2d = tsne.fit_transform(emb)
        fig = px.scatter(x=emb_2d[:,0], y=emb_2d[:,1], text=subset['ficheiro'], title=f't-SNE ({modelo_tsne})')
        fig.show()
    except Exception as e:
        print(f'Erro no t-SNE: {e}')
else:
    print('Dados insuficientes para gerar t-SNE.')

### PROCURA REAL OU ADHOC NA BASE DE DADOS MILVUS... VAMOS USAR SEMPRE A BASE DE DADOS COM MAIS DADOS INSERIDOS... 

AD HOC SEARCH


-TODO FIX ADHOC SEARCH WITH YAMMET AND VGGISH

In [18]:
# 📤 Pesquisa Ad-hoc (Upload ou Ficheiro da Base de Dados) e funções auxiliares
import tempfile
import librosa
import numpy as np
import torch
import pandas as pd
import plotly.express as px
import ast
import openl3
import io
import os

# Import transformers models
from transformers import (
    Wav2Vec2Processor, Wav2Vec2Model,
    ClapProcessor, ClapModel,
    ASTFeatureExtractor, ASTModel
)

import tensorflow as tf
import tensorflow_hub as hub

# Função para extrair embedding de um áudio
def extract_embedding(model_name, temp_path):
    audio, sr = librosa.load(temp_path, sr=None)
    
    if model_name == "wav2vec2":
        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        # Resample if needed
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()[0]
    
    elif model_name == "clap":
        processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
        model = ClapModel.from_pretrained("laion/clap-htsat-unfused")
        # Resample if needed
        if sr != 48000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=48000)
        inputs = processor(audios=audio, sampling_rate=48000, return_tensors="pt")
        with torch.no_grad():
            embeddings = model.get_audio_features(**inputs)
        return embeddings[0].numpy()
    
    elif model_name == "ast":
        fe = ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
        model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
        # Resample if needed
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        inputs = fe(audio, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()[0]
    
    elif model_name == "vggish":
        # Mesmo código usado no Streamlit que funciona
        import tensorflow_hub as hub
        # Resample para 16kHz conforme exigido pelo VGGish
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        # Converter para mono se for estéreo
        if len(audio.shape) > 1:
            audio = np.mean(audio, axis=1)
        # Carregar modelo e obter embedding
        vggish_model = hub.load('https://tfhub.dev/google/vggish/1')
        embedding = vggish_model(audio).numpy()
        # Fazer média se tivermos múltiplos embeddings
        if embedding.ndim > 1:
            embedding = np.mean(embedding, axis=0)
        return embedding
    
    elif model_name == "yamnet":
        # Mesmo código usado no Streamlit que funciona
        import tensorflow_hub as hub
        import tensorflow as tf
        # Resample para 16kHz conforme exigido pelo YAMNet
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        # Converter para tensor do TensorFlow
        waveform = tf.convert_to_tensor(audio, dtype=tf.float32)
        # Carregar modelo e obter embedding
        yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
        scores, embeddings, spectrogram = yamnet_model(waveform)
        # Fazer média dos embeddings se tivermos múltiplos
        embedding = np.mean(embeddings.numpy(), axis=0)
        return embedding
    
    elif model_name == "openl3":
        audio = np.mean(audio, axis=1) if audio.ndim > 1 else audio
        emb, _ = openl3.get_audio_embedding(audio, sr, content_type="music", embedding_size=512)
        return emb.mean(axis=0) if emb.ndim > 1 else emb.squeeze()
    
    else:
        raise ValueError("Modelo não suportado")

# Função para pesquisa real a partir do banco de dados
def real_search_from_db(df, client, modelo_milvus, collection_name, k):
    dimensoes_modelos = {
        "ast": 768, "clap": 512, "yamnet": 1024,
        "openl3": 512, "vggish": 128, "wav2vec": 768, "wav2vec2": 768,
    }
    dim_esperada = dimensoes_modelos.get(modelo_milvus)
    if not dim_esperada:
        raise ValueError(f"Dimensão desconhecida para modelo '{modelo_milvus}'")

    ficheiros = df[(df['modelo'] == modelo_milvus) & (df['embedding'].apply(lambda x: len(x) == dim_esperada))]['ficheiro'].tolist()
    if not ficheiros:
        raise ValueError(f"Sem ficheiros com dimensão {dim_esperada} para modelo '{modelo_milvus}'")

    row_query = df[(df['modelo'] == modelo_milvus) & (df['ficheiro'] == ficheiros[0])].iloc[0]
    embedding_query = row_query['embedding']

    result = client.search(
        collection_name=collection_name,
        data=[embedding_query],
        limit=k,
        output_fields=["id", "vector"]
    )
    hits = result[0] if result else []
    res_df = pd.DataFrame([{"id": h.get("id"), "distance": h.get("distance")} for h in hits])
    return res_df

# Interface de usuário
upload_widget = widgets.FileUpload(accept='.wav,.mp3,.flac,.ogg', multiple=False)
modelo_options = ["wav2vec2", "clap", "ast", "vggish", "yamnet", "openl3"]
modelo_widget = widgets.Dropdown(options=modelo_options, description="Modelo")
usar_ficheiro_existente = widgets.Checkbox(value=False, description="Usar ficheiro do dataset")
ficheiro_select = widgets.Dropdown(description="Ficheiro", options=[])

# Widget para seleção de coleção
collection_widget = widgets.Dropdown(description="Coleção", options=[])
usar_colecao_padrao = widgets.Checkbox(value=True, description="Usar coleção padrão")

output = widgets.Output()

# Função para atualizar opções de coleção
def update_collection_options(*args):
    try:
        collections = client.list_collections() if client else []
        if collections:
            collection_widget.options = collections
    except Exception as e:
        with output:
            print(f"Erro ao atualizar coleções: {e}")

# Função para atualizar opções de arquivo
def update_file_options(*args):
    if usar_ficheiro_existente.value:
        ficheiro_select.options = df[df['modelo'] == modelo_widget.value]['ficheiro'].tolist()
        ficheiro_select.layout.display = 'block'
    else:
        ficheiro_select.layout.display = 'none'
        # Exibe aviso para VGGish e YAMNet quando esses modelos são selecionados
        modelo_atual = modelo_widget.value

# Função para atualizar visibilidade do seletor de coleção
def update_collection_visibility(*args):
    if usar_colecao_padrao.value:
        collection_widget.layout.display = 'none'
    else:
        collection_widget.layout.display = 'block'
        # Atualize as opções quando o widget ficar visível
        update_collection_options()

def perform_search(b):
    with output:
        output.clear_output()
        try:
            if usar_ficheiro_existente.value:
                if not ficheiro_select.value:
                    print("Por favor, selecione um ficheiro do dataset.")
                    return
                    
                # Usar ficheiro do dataset
                row = df[(df['modelo'] == modelo_widget.value) & (df['ficheiro'] == ficheiro_select.value)].iloc[0]
                embedding = np.array(row['embedding'])
                print(f"Usando embedding do ficheiro: {ficheiro_select.value}")
            
            else:
                # Este bloco estava dentro do if anterior, corrigindo a indentação
                if not upload_widget.value:
                    print("Por favor, carregue um ficheiro de áudio.")
                    return
                
                # Usar ficheiro carregado
                file_content = upload_widget.value[0]['content']
                file_bytes = io.BytesIO(file_content)
                file_name = upload_widget.value[0]['name']
                
                with tempfile.NamedTemporaryFile(delete=False, suffix=f"_{file_name}") as f:
                    f.write(file_bytes.read())
                    temp_path = f.name
                
                print(f"Extraindo embedding com modelo {modelo_widget.value}...")
                embedding = extract_embedding(modelo_widget.value, temp_path)
                os.unlink(temp_path)  # Remove o arquivo temporário
            
            # Determinar qual coleção usar
            if usar_colecao_padrao.value:
                collection_name = f"audio_{modelo_widget.value}"
            else:
                collection_name = collection_widget.value
                if not collection_name:
                    print("Por favor, selecione uma coleção.")
                    return
            
            # Verificar se a coleção existe
            if collection_name not in client.list_collections():
                print(f"Coleção {collection_name} não existe no Milvus!")
                return
                
            print(f"Pesquisando na coleção {collection_name}...")
            result = client.search(
                collection_name=collection_name,
                data=[embedding.tolist()],
                limit=5,
                output_fields=['id', 'vector']
            )
            
            hits = result[0] if result else []
            if not hits:
                print("Nenhum resultado encontrado.")
                return
                
            df_res = pd.DataFrame([{'id': h.get('id'), 'distance': h.get('distance')} for h in hits])
            display(df_res)
            
            # Visualização dos resultados
            fig = px.bar(df_res, x='id', y='distance', title=f'Resultados da Pesquisa - Coleção: {collection_name}')
            fig.show()
            
        except NotImplementedError as e:
            print(f"⚠️ {str(e)}")
        except Exception as e:
            import traceback
            print(f"Erro ao processar ou pesquisar: {str(e)}")
            traceback.print_exc()

# Configuração da interface
search_button = widgets.Button(description="Pesquisar")
search_button.on_click(perform_search)
usar_ficheiro_existente.observe(update_file_options, names='value')
modelo_widget.observe(update_file_options, names='value')
usar_colecao_padrao.observe(update_collection_visibility, names='value')

# Inicializar visibilidade
ficheiro_select.layout.display = 'none'
collection_widget.layout.display = 'none'  # Esconde o seletor de coleção inicialmente

# Atualizar coleções disponíveis no início
update_collection_options()

# Exibir widgets
display(widgets.VBox([
    widgets.HBox([modelo_widget, usar_ficheiro_existente]),
    widgets.HBox([usar_colecao_padrao, collection_widget]),
    upload_widget,
    ficheiro_select,
    search_button,
    output
]))

# Inicializar opções de ficheiros
update_file_options()